In [2]:
!pip install datasets
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from datasets import Dataset
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import torch.nn as nn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [73]:
data = "I will kill myself. Iam doing nothing right with this life of mine. it just sucks so bad"


In [74]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.add('im')

def text_clean(txt):
  txt = txt.lower()
  cleaned_text = re.sub(r'[@#][^ \t\r\n]+', '', txt)
  tokens = cleaned_text.split()
  filter = [w for w in tokens if not w in stop_words]
  url = re.compile(r'https?://\S+|www\.\S+')

  for each_word in range(len(filter)):
    filter[each_word] = url.sub(r'', filter[each_word])
    filter[each_word] = re.sub("\'", "", filter[each_word])

  clean = " ".join(filter)

  return clean

data = text_clean(data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [53]:
def tokenize(text, max_len=512):
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=max_len,
        pad_to_max_length=True,
        truncation = True,
        return_token_type_ids=True
    )

    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs["token_type_ids"]

    return {
        'ids': torch.tensor(ids, dtype=torch.long),
        'mask': torch.tensor(mask, dtype=torch.long),
        'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
    }

In [75]:
tokenized_data = tokenize(data)

In [24]:
!gdown '1LD2QID0EEZP9K-3cq4dKyZIKVPdsc9ge'

Downloading...
From (original): https://drive.google.com/uc?id=1LD2QID0EEZP9K-3cq4dKyZIKVPdsc9ge
From (redirected): https://drive.google.com/uc?id=1LD2QID0EEZP9K-3cq4dKyZIKVPdsc9ge&confirm=t&uuid=7e2a7dd0-1e53-4725-8297-63bbbc204c96
To: /content/distlbert_mental_classifier.pt
100% 268M/268M [00:01<00:00, 135MB/s]


In [26]:
class DistilBERT(nn.Module):
  def __init__(self):
        super(DistilBERT, self).__init__()

        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = nn.Linear(768, 768)
        #self.dropout = nn.Dropout(0.4)
        self.dropout = nn.Dropout(0.4)
        self.classifier = nn.Linear(768, 4)

  def forward(self, input_ids, attention_mask, token_type_ids):
      output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
      hidden_state = output_1[0]
      pooler = hidden_state[:, 0]
      pooler = self.pre_classifier(pooler)
      pooler = nn.Tanh()(pooler)
      pooler = self.dropout(pooler)
      output = self.classifier(pooler)
      return output

model = DistilBERT()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [28]:
PATH = "/content/distlbert_mental_classifier.pt"
model = torch.load(PATH, map_location=torch.device('cpu'))
model.eval()

DistilBERT(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

In [70]:
def predict(data):
    model.eval()

    with torch.no_grad():
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        outputs = model(ids, mask, token_type_ids)
        softmax_probs = torch.softmax(outputs.data, dim=1)
        pred_prob, pred_label = torch.max(softmax_probs, dim=1)
    return pred_label

In [71]:
pred = predict(tokenized_data)

In [77]:
def pipeline(data):
  data = text_clean(data)
  data = tokenize(data)
  return predict(data)

In [76]:
num2label = {0 : "neutral", 1 : "sad", 2 : "anxiety", 3 : "depressed"}

In [132]:
txt = "My boyfriend beats me and abuses me"
pred = pipeline(txt)

In [133]:
print(txt)
print("Prediction: ", num2label[pred.item()])

My boyfriend beats me and abuses me
Prediction:  sad
